In [ ]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
pd.set_option('display.max_rows', 10)

%load_ext autoreload
%autoreload 2

# Potential impact of post-exposure prophalaxis for COVID-19 with monoclonal antibodies

Updated from March 2021 analysis to include more recent confirmed cases, and also vaccination coverage.

Also updated to use completed, and now peer-reviewed, results from RCT

And in this version, use the KFF survey results that 77% of unvaccinated adults say everyone in their household is unvaccinated. https://www.kff.org/coronavirus-covid-19/poll-finding/kff-covid-19-vaccine-monitor-june-2021/

In [ ]:
# range of values to consider in alternative scenarios
coverage_levels = [0, 1/4, 1/2, 3/4, 1]
age_lower_bounds = [0, 20, 40, 50, 60, 70, 80]

In [ ]:
n_draws = 100 # number of bootstrap resamples for quantifying uncertainty
np.random.seed(12345)  # set random seed for reproducibility

In [ ]:
attack_rate_without_mAbs = np.random.binomial(752, 59/752, size=n_draws) / 752
attack_rate_with_mAbs = np.random.binomial(753, 11/753, size=n_draws) / 753

mAbs_unit_cost = np.random.normal(1_250, 125, size=n_draws)  # 1_250 is what it says online;
COVID_hospitalization_unit_cost = np.random.normal(73_300, 7_330, size=n_draws)

In [ ]:
%%time

# Administrated data on how many vaccines have been given in the US
url = "https://data.cdc.gov/api/views/rh2h-3yt2/rows.csv?accessType=DOWNLOAD"
vax_dt = pd.read_csv(url)
#vax_dt

In [ ]:
total_vax = vax_dt[(vax_dt["Date"] == "05/31/2021") & (vax_dt["date_type"] == "Report") & (vax_dt["Location"] == "US")]
us_pop = 332_378_911 #census population on 5/31/2021 (https://www.census.gov/popclock/)

p_vax   = np.random.binomial(us_pop, total_vax.Admin_Dose_1_Cumulative/us_pop, size=n_draws) / us_pop
p_unvax = 1-p_vax
#p_vax, p_unvax

In [ ]:
# TODO: fill in the following with an appropriate RCT (or meta-analysis, or real-world effectiveness study...)
# numbers from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8022879/pdf/mm7013e3.pdf
n_treated = 2_928 #(excluded 33 cases in study because occured before 14 days period)
cases_treated = 11

n_control = 989
cases_control = 161

efficacy = 1 - (
    (np.random.binomial(n_treated, cases_treated/n_treated, size=n_draws) / n_treated)
    /
    (np.random.binomial(n_control, cases_control/n_control, size=n_draws) / n_control))

## equ1: p_cc = (p_cc_vax*p_vax) + (p_cc_unvax*p_unvax)
## equ2: p_unvax_cc = (p_cc_unvax * p_unvax)/p_cc
## equ3: p_cc_vax = efficacy * p_cc_unvax

## substitute equ3 into equ1: p_cc = ((efficacy * p_cc_unvax) * p_vax) + (p_cc_unvax*p_unvax)
## p_cc = p_cc_unvax((efficacy * p_vax) + p_unvax)

## substitute equ1 into equ2: p_unvax_cc = (p_cc_unvax * p_unvax) / p_cc_unvax((efficacy * p_vax) + p_unvax)
p_unvax_cc =  p_unvax / (((1-efficacy) * (p_vax)) + p_unvax)
#p_unvax_cc

In [ ]:
pr_unvaccinated_cond_on_test_positive = p_unvax_cc  # np.random.binomial(10_000, unvac_covPos, size=n_draws)/10_000 

# https://files.kff.org/attachment/Topline-KFF-COVID-19-Vaccine-Monitor-June-2021.pdf
# n = 458, p_all_hh_members unvaccinated = 0.69
pr_case_in_unvaccinated_household = pr_unvaccinated_cond_on_test_positive * np.random.binomial(458, 0.69, size=n_draws) / 458

In [ ]:
%%time

df_cdc = pd.read_csv('data/cdc_covid_linelist_may_2021.csv.bz2', low_memory=False)

# 1.	Estimate the number of COVID-19 case count, n_1, and the age-/sex-/race-stratified case counts

    n_1 (a,s,r)=# cases for strata (a,s,r).


In [ ]:
# first select all cases during time period of interest
diagnosis_date = pd.to_datetime(df_cdc.pos_spec_dt)
rows = (diagnosis_date >= '2021-05-01') & (diagnosis_date < '2021-06-01')
t = df_cdc[rows].copy()  # NOTE: same dataframe, code just kept for convenience

In [ ]:
n_1_total = len(t) #total cases
print(f'Total cases: {n_1_total:,}')

In [ ]:
age_cutpoints = [0,10,20,30,40,50,60,70,80,125]
cdc_age_map = {f'{a} - {a+9} Years':a for a in age_cutpoints}
cdc_age_map['80+ Years'] = 80
t['age'] = t.age_group.map(cdc_age_map)

t['age_group'] = pd.cut(t.age, age_cutpoints, right=False)
t['sex'] = t.sex.map({'Male': 'male', 'Female':'female'})
t['race_eth'] = t.race_ethnicity_combined.map({
    'White, Non-Hispanic':'White',
    'Black, Non-Hispanic':'Black',
    'Hispanic/Latino':'Hispanic', 
    'American Indian/Alaska Native, Non-Hispanic':'Other',
    'Multiple/Other, Non-Hispanic':'Other',
    'Asian, Non-Hispanic':'Other',
    'Native Hawaiian/Other Pacific Islander, Non-Hispanic':'Other',
})

t['weight'] = 1.0
#t

In [ ]:
df_cdc_month = t
#df_cdc_month.shape

In [ ]:
# stratification factors
age_groups = pd.cut(np.random.uniform(0, 125, 10_000), age_cutpoints, right=False).unique().sort_values()
#age_groups = pd.cut(list(range(0, 125)), age_cutpoints, right=False).unique()
sexes = ['male', 'female']
race_eth = ['White', 'Black', 'Hispanic', 'Other']
stratification_groups = pd.MultiIndex.from_product([age_groups, sexes, race_eth])

In [ ]:
%%time
#    n_1 (a,s,r) = number of cases for strata (a,s,r).

n_1 = {}

for k in range(n_draws):
    # use bootstrap to resample confirmed cases, to quantify uncertainty
    resampled_rows = np.random.choice(df_cdc_month.index, size=len(t), replace=True)
    df_selected_cdc = df_cdc_month.loc[resampled_rows]
    
    n_1[k] = df_selected_cdc.groupby(['age_group', 'sex', 'race_eth']).weight.sum()
    
    # use parametric resampling from a possion distribution to also include uncertainty in total number of cases
    n_1[k] = np.random.poisson(n_1_total) * n_1[k] / n_1[k].sum()

In [ ]:
# TODO: Table 1 showing the demographics of the confirmed case population,
# highlighting amount of missingness and also including household size distribution

In [ ]:
n_1_mean = pd.DataFrame(n_1).T.mean()
np.round(n_1_mean.unstack().unstack(),1)  # number of confirmed cases of COVID-19, stratified by age, sex, and race/ethnicity

In [ ]:
# assert np.allclose(n_1_mean.sum(), n_1_total)


## 2. Estimate the stratified number of household contacts who might benefit from mAbs PEP 

Think of entry $(i,j)$ in this table as the number of people in group $j$ you would find who have been exposed to COVID, if you did a household visit for a COVID-19 case in group $i$.

### TODO: Figure out how to quantify uncertainty for this in a reasonable amount of time.

In [ ]:
acs = pd.read_csv('data/acs_2019_pums.csv.bz2')
#acs.head()

In [ ]:
acs['age_group'] = pd.cut(acs.age, age_cutpoints, right=False)
acs['sex'] = acs.sex.map({1:'male', 2:'female'})
acs['race_eth'] = acs.race_eth.map({1:'White',
                                    2:'Black',
                                    3:'Hispanic', 
                                    4:'Other'})
#acs.head()

In [ ]:
g_acs = acs[acs.household_id.str.contains('HU')].groupby(['age_group', 'sex', 'race_eth'])
#g_acs.head()

In [ ]:
%%time
n_hh = {}

for k in range(n_draws):
    print('\n',k)
    n_hh[k] = pd.DataFrame(0, index=stratification_groups, columns=stratification_groups)
    n_hh[k] = n_hh[k].sort_index()  # make a pesky warning go away, and possibly make things faster --- see https://stackoverflow.com/questions/54307300/what-causes-indexing-past-lexsort-depth-warning-in-pandas
    
    for i in stratification_groups:
        print('.', flush=True, end=' ')
        hh_ids = g_acs.get_group(i).household_id  # ids for households containing a person from stratification group i
        hh_ids_k = np.random.choice(hh_ids, size=100, replace=True)  # randomly resampled hhs with person from group i

        for j in stratification_groups:
            t = g_acs.get_group(j) # all people in stratification from j

            df_ij = t[t.household_id.isin(hh_ids_k)]  # people in stratification group j who live in household with person from stratification group i

            hh_ij = pd.Series(0, index=hh_ids_k).add(
                        df_ij.household_id.value_counts() * pd.Series(hh_ids_k).value_counts(),  # resampling with replacement means some hhs will appear more than once
                        fill_value=0
                    )

            n_hh[k].loc[i,j] = hh_ij.mean()
            if i == j:
                n_hh[k].loc[i,j] -= 1 # don't include the person with a confirmed case when tallying household exposure count

In [ ]:
%%time

n_2_unvax = {}
n_2_unvax_total = {}

n_2_all = {}
n_2_all_total = {}

for k in range(n_draws):
    n_2_unvax[k] = pd.Series(0.0, index=stratification_groups)
    n_2_all[k] = pd.Series(0.0, index=stratification_groups)
    for j in stratification_groups:
        a0,a1 = j[0].left, j[0].right
        s,r=j[1],j[2]
        j_cov = (f'[{a0}, {a1})', s, r)
        for i in stratification_groups:
            n_2_unvax[k][j] += n_1[k][i] * pr_case_in_unvaccinated_household[k] * n_hh[k].loc[i,j] # count only the unvaccinated who's household is all unvaccinated, which is a lower-bound on total
            n_2_all[k][j] += n_1[k][i] * n_hh[k].loc[i,j]
    n_2_unvax_total[k] = n_2_unvax[k].sum()
    n_2_all_total[k] = n_2_all[k].sum()
n_2_unvax_mean = sum(n_2_unvax.values()) / n_draws
n_2_all_mean = sum(n_2_all.values()) / n_draws
np.round(n_2_all_mean.unstack().unstack(), 1)

In [ ]:
np.round(n_2_unvax_mean.unstack().unstack(), 1)

In [ ]:
n_2_all_total_mean = sum(n_2_all_total.values()) / n_draws
n_2_all_total_lb = np.percentile(list(n_2_all_total.values()), 2.5)
n_2_all_total_ub = np.percentile(list(n_2_all_total.values()), 97.5)

In [ ]:
print(f'total number of (vaccinated & unvaccinated) household contacts: {n_2_all_total_mean:,.1f} (95% UI {n_2_all_total_lb:,.1f}-{n_2_all_total_ub:,.1f})')

In [ ]:
n_2_total_mean = sum(n_2_unvax_total.values()) / n_draws
n_2_total_lb = np.percentile(list(n_2_unvax_total.values()), 2.5)
n_2_total_ub = np.percentile(list(n_2_unvax_total.values()), 97.5)

In [ ]:
print(f'total number of unvaccinated household contacts: {n_2_total_mean:,.1f} (95% UI {n_2_total_lb:,.1f}-{n_2_total_ub:,.1f})')

In [ ]:
# household exposure rate (exposures per 100,000 person-months) by race
n_2_race = {}
for k in range(n_draws):
    n_2_race[k] = 100_000 * (n_2_unvax[k].unstack().sum() / g_acs.weight.sum().unstack().sum())

np.round(sum(n_2_race.values())/n_draws, 1)

## 3. Estimate the number of people who receive PEP in scenarios with a range of coverage levels c and minimum age a_0 for receiving PEP

In [ ]:
n_PEP = {}
for c in coverage_levels:
    for a0 in age_lower_bounds:
        n_PEP[c,a0] = {}
        for k in range(n_draws):
            age_mask = pd.Series([a.left >= a0 for a,s,r in stratification_groups],
                         index=stratification_groups)
            n_PEP[c,a0][k] = c*n_2_unvax[k]*age_mask


In [ ]:
# treatment rate (tx per 100,000 person-months) by race
c,a = .50,50
n_PEP_mean = sum(n_PEP[c, a].values())/n_draws
np.round(100_000 * (n_PEP_mean.unstack().sum() / g_acs.weight.sum().unstack().sum()), 1) 

## 4.	Estimate the number of people who develop a symptomatic COVID-19 infection in each scenario

In [ ]:
n_COVID = {}
for c in coverage_levels:
    for a0 in age_lower_bounds:
        n_COVID[c,a0] = {}
        for k in range(n_draws):
            n_COVID[c,a0][k] = attack_rate_without_mAbs[k] * (n_2_unvax[k] - n_PEP[c,a0][k]) \
                        + attack_rate_with_mAbs[k] * n_PEP[c,a0][k]

In [ ]:
n_COVID_mean = sum(n_COVID[0,70].values())/n_draws

np.round(n_COVID_mean.unstack().unstack(),1)

In [ ]:
n_COVID_mean = sum(n_COVID[.5,70].values())/n_draws

np.round(n_COVID_mean.unstack().unstack(),1)

In [ ]:
print(f'confirmed cases: {n_1_total:,.0f}\npredicted hh infections: {sum(n_COVID[0,0].values()).sum()/n_draws:,.0f}')

In [ ]:
# secondary infection rate (symptomatic infections per 100,000 person-months) by race
c,a = .50,50
n_COVID_mean = sum(n_COVID[c,a].values())/n_draws

np.round(100_000 * (n_COVID_mean.unstack().sum() / g_acs.weight.sum().unstack().sum()), 1)

In [ ]:
# averted secondary infection rate (symptomatic infections per 100,000 person-months) by race
c,a = .50,50
n_COVID_averted_mean = sum([n_COVID[0, 0][k] - n_COVID[c, a][k] for k in range(n_draws)]) / n_draws
averted_symptomatic_race = np.round(10_000_000 * (n_COVID_averted_mean).unstack().sum() / g_acs.weight.sum().unstack().sum(), 0).apply(int)

# 5.	Estimate the number of hospitalizations in each scenario

[Rate Ratios for Hospitalization by Age](https://www.cdc.gov/coronavirus/2019-ncov/covid-data/investigations-discovery/hospitalization-death-by-age.html)

But better to calculate it myself

In [ ]:
rows = df_cdc_month.hosp_yn.isin(['No', 'Yes'])
df_cdc_month_complete_cases = df_cdc_month[rows].copy()

df_cdc_month_complete_cases['hosp'] = (df_cdc_month_complete_cases.hosp_yn == 'Yes').astype(float)
# np.round(100 * df_cases.groupby('age_group').hosp.mean(), 1)

In [ ]:
hospitalization_rate = {}

for k in range(n_draws):
    resampled_rows = np.random.choice(df_cdc_month_complete_cases.index, size=len(df_cdc_month_complete_cases))
    t = df_cdc_month_complete_cases.loc[resampled_rows]
    hospitalization_rate[k] = pd.Series(t.groupby(['age_group', 'sex', 'race_eth']).hosp.mean(), index=stratification_groups)

hospitalization_rate_mean = sum(hospitalization_rate.values()) / n_draws
np.round(hospitalization_rate_mean.unstack().unstack()*100,1)

In [ ]:
# this is hospitalization rate among *confirmed cases*
# is the attack rate from Ruanne's study for "symptomatic cases"? And how many of these are confirmed
for k in range(n_draws):
    hospitalization_rate[k] *= 1 

In [ ]:
%%time

n_hosp = {}
for c in coverage_levels:
    for a0 in age_lower_bounds:
        n_hosp[c,a0] = {}
        for k in range(n_draws):
            n_hosp[c,a0][k] = hospitalization_rate[k] * n_COVID[c,a0][k]

In [ ]:
(n_hosp[0, 0][0] - n_hosp[.5, 50][0]).unstack().unstack()

In [ ]:
# hospitalization rate (tx per 100,000 person-months) by race
c,a = .50,50
n_hosp_mean = sum(n_hosp[c, a].values()) / n_draws
np.round(100_000 * (n_hosp_mean.unstack().sum() / g_acs.weight.sum().unstack().sum()), 1) 

In [ ]:
# averted hospitalization rate (hospitalizations per 10,000,000 person-months) by race
n_hosp_averted_mean = sum([n_hosp[0, 0][k] - n_hosp[c, a][k] for k in range(n_draws)]) / n_draws
averted_hosp_race = np.round(10_000_000 * (n_hosp_averted_mean.unstack().sum() / g_acs.weight.sum().unstack().sum()), 0).apply(int) 
averted_hosp_race

# 6. Find number of deaths

In [ ]:
df_cdc_month_complete_cases.death_yn.value_counts()

In [ ]:
df_cdc_month_complete_cases['death'] = df_cdc_month_complete_cases.death_yn.map({'Yes':1, 'No':0})
hosp_rows = df_cdc_month_complete_cases[df_cdc_month_complete_cases.hosp == 1].index

hospitalization_fatility_ratio = {}
for k in range(n_draws):
    resampled_rows = np.random.choice(hosp_rows, size=len(hosp_rows), replace=True)
    t = df_cdc_month_complete_cases.loc[resampled_rows]
    hospitalization_fatility_ratio[k] = t.groupby(['age_group', 'sex', 'race_eth']).death.mean()
    hospitalization_fatility_ratio[k] = pd.Series(hospitalization_fatility_ratio[k], index=stratification_groups)
np.round((sum(hospitalization_fatility_ratio.values()) / n_draws).unstack().unstack()*100, 1)

In [ ]:
n_deaths = {}
for c in coverage_levels:
    for a0 in age_lower_bounds:
        n_deaths[c,a0] = {}
        for k in range(n_draws):
            n_deaths[c,a0][k] = hospitalization_fatility_ratio[k] * n_hosp[c,a0][k]

In [ ]:
c,a = .50,50

# averted death rate (hospitalizations per 100,000 person-months) by race
n_death_averted_mean = sum([n_deaths[0, 0][k] - n_deaths[c, a][k] for k in range(n_draws)]) / n_draws
np.round((n_death_averted_mean).unstack().unstack(),1)

In [ ]:
# averted death rate (deaths per 100,000 person-months) by race
n_death_mean = sum([n_deaths[c, a][k] for k in range(n_draws)]) / n_draws
np.round(100_000 * (n_death_mean.unstack().sum() / g_acs.weight.sum().unstack().sum()), 3) 

In [ ]:
# averted death rate (deaths averted per 10,000,000 person-months) by race
averted_death_race = np.round(100_000_000 * ((n_death_averted_mean).unstack().sum() / g_acs.weight.sum().unstack().sum()), 0).apply(int)
averted_death_race

## 7.	Estimate the cost of administering mAbs and the cost of COVID-19 hospitalizations 

Tough [to get cost of mAbs](https://www.npr.org/sections/health-shots/2020/10/28/928841997/government-signs-deal-for-covid-19-treatments-from-eli-lilly)

Lots of hits from a search of [cost of hospitalization](https://www.fairhealth.org/article/costs-for-a-hospital-stay-for-covid-19).

In [ ]:
n_hosp[c,a0][k].sum()

In [ ]:
doses = {}
mAbs_cost = {}
hosp_cost = {}
for c in coverage_levels:
    for a0 in age_lower_bounds:
        doses[c,a0] = {}
        mAbs_cost[c,a0] = {}
        hosp_cost[c,a0] = {}
        for k in range(n_draws):
            doses[c,a0][k] = n_PEP[c,a0][k].sum()
            mAbs_cost[c,a0][k] = n_PEP[c,a0][k].sum() * mAbs_unit_cost[k]
            hosp_cost[c,a0][k] = n_hosp[c,a0][k].sum() * COVID_hospitalization_unit_cost[k]

In [ ]:
pd.DataFrame(mAbs_cost).mean()

In [ ]:
def print_mean_and_ui(z):
    print(f'{np.mean(z):,.0f} ({np.percentile(z, 2.5):,.0f} - {np.percentile(z, 97.5):,.0f})')          
    
c,a = .50,50

print('averted cost')
print_mean_and_ui(
    [((mAbs_cost[0, 0][k]+hosp_cost[0, 0][k]) - (mAbs_cost[c, a][k]+hosp_cost[c, a][k])).sum() for k in range(n_draws)]
)

In [ ]:
cost = pd.DataFrame(mAbs_cost).mean() + pd.DataFrame(hosp_cost).mean()
(cost.unstack() / 1_000_000).plot(marker='o')
plt.ylabel('Cost (Million USD)')
plt.grid();

In [ ]:
pd.DataFrame(doses).mean().unstack().plot(marker='o')
# plt.semilogy()
plt.ylabel('mAbs doses');

# Burden Averted

In [ ]:
#(pd.DataFrame(n_COVID[0,0]).mean().sum() - pd.Series({i: pd.DataFrame(n_COVID[i]).mean().sum() for i in n_COVID}))

In [ ]:
#print('averted secondary infections')
def mean_ui(df, c, a, div = 1):
    # shows cases averted with UIs
    z = [(df[0, 0][k] - df[c, a][k]).sum() for k in range(n_draws)]
    return f'{np.mean(z)/div:,.0f} ({np.percentile(z, 2.5)/div:,.0f} - {np.percentile(z, 97.5)/div:,.0f})'
    
mean_ui(n_COVID, .50, 50)

In [ ]:
def averted(df, div = 1):
    pretty_table_ui = {}
    pretty_table    = {}
    
    for c in coverage_levels:
        for a in age_lower_bounds:
            pretty_table_ui[c, a] = mean_ui(df, c, a, div)
            pretty_table[c, a] = np.mean([(df[0, 0][k] - df[c, a][k]).sum() for k in range(n_draws)])
    
    return pd.Series(pretty_table), pd.Series(pretty_table_ui)
#averted(n_COVID)

In [ ]:
symptomatic_infections_averted = averted(n_COVID)[1]
hospitalizations_averted = averted(n_hosp)[1]
deaths_averted = averted(n_deaths)[1]
doses_averted = averted(doses)[1]

In [ ]:
pd.set_option('display.max_rows', 35)

pd.DataFrame({
    'Symptomatic Infections Averted': symptomatic_infections_averted,
    'Hospitalizations Averted': hospitalizations_averted,
    'Deaths Averted': deaths_averted,
    'Doses': doses_averted, #pd.DataFrame(doses).mean()/1e3
    'Incremental Cost':((cost-cost.loc[0,0])/1e6).round(1)
}).unstack().unstack().unstack().loc[:, [0.25, 0.50, 0.75, 1]]

In [ ]:
ICER = (cost-cost.loc[0,0]) / (averted(n_deaths)[0])
np.round((ICER / 1_000).unstack(), 1)

In [ ]:
(ICER / 1_000).unstack().T.iloc[:,-1].plot(marker='o')
plt.ylabel('ICER (Thousand USD per death averted)')
plt.xlabel('Minimum age for PEP (years)')
# plt.semilogy();

# PEP with mAbs is a benefit for patients and for hospitals

In [ ]:
def summary_results(coverage_level, age_lower_bound):
    n_symp_averted = symptomatic_infections_averted.loc[coverage_level, age_lower_bound]
    n_hosp_averted = hospitalizations_averted.loc[coverage_level, age_lower_bound]
    n_deaths_averted = deaths_averted.loc[coverage_level, age_lower_bound]
    unit_cost = mAbs_unit_cost
    incremental_cost = (cost.loc[coverage_level, age_lower_bound]-cost.loc[0,0])

    result_str = f"""Results: In a month of similar intensity to May, 2021, in USA, a program reaching {coverage_level*100:.0f}% 
    of exposed household members aged {age_lower_bound}+, would avert {n_symp_averted} symptomatic infections,
    {n_hosp_averted} hospitalizations, and {n_deaths_averted} deaths.  If the unit cost of administering the intervention 
    was {pd.Series(unit_cost).mean():,.0f} dollars, this program would save the health system {-incremental_cost:,.0f} dollars."""
    
    print(result_str)
summary_results(1/2, 50)

In [ ]:
summary_results(1/2, 20)

In [ ]:
summary_results(1/2, 80)

# Number-plugging values

In [ ]:
print(f"""[[summary of data: RCT included X subjects (Y in treatment group and Z in control),
which we combined with confirmed cases data on {len(df_cdc_month):,.0f} individuals and
household structure data derived from {len(acs):,.0f} individuals in {acs.household_id.nunique():,.0f} households.]]
""")

In [ ]:
n_2 = pd.DataFrame(n_2_unvax)
#n_2

In [ ]:
print(f"""[[Confirmed case and household structure results,
summarizing something about the racial segregation of household structure,
the age-assortativity. In [[May]], there were [[{len(df_cdc_month):,.0f}]]
confirmed cases of COVID-19, which we estimate resulted
in {n_2.sum().mean():,.0f} (95% UI {np.percentile(n_2.sum(), 2.5):,.0f}-{np.percentile(n_2.sum(), 97.5):,.0f}) individuals with household exposure to COVID-19. ]] [[stratify counts or rates by race??]]""")

In [ ]:
unvax_race = n_2.unstack().sum().unstack().mean()
unvax_race

In [ ]:
unvax_race_per = unvax_race/sum(unvax_race) *100
unvax_race_per

In [ ]:
#Resut paragraph 1
print(f"""In May, there were {len(df_cdc_month):,.0f} confirmed cases of COVID-19 in the US, which we estimate resulted in at least {n_2_total_mean:,.0f} (95% UI {n_2_total_lb:,.0f}-{n_2_total_ub:,.0f}) unvaccinated individuals with household exposure to COVID-19. Using the racial and ethnic composition of households, COVID-19 cases, and vaccine coverage estimates described above, we estimated unvaccinated household contacts to include {unvax_race['Black']:,.0f} ({unvax_race_per['Black']:,.1f}%) non-Hispanic Black individuals, {unvax_race['Hispanic']:,.0f} ({unvax_race_per['Hispanic']:,.1f}%) Hispanic individuals, {unvax_race['White']:,.0f} ({unvax_race_per['White']:,.1f}%) non-Hispanic white individuals, and {unvax_race['Other']:,.0f} ({unvax_race_per['Other']:,.1f}%) members of other non-Hispanic racial and ethnic groups.""")


In [ ]:
print(f"""[[Number who receive PEP in alternative scenarios, and for alternative age cutoffs. Perhaps in a table?  Population-level rates of treatment stratified by race?]]""")

In [ ]:
def my_summary(x):
    mean = np.mean(x)
    lb = np.percentile(x, 2.5)
    ub = np.percentile(x, 97.5)
    results = locals()
    results.pop('x')
    for k in results:
        results[k] = '{:,.0f}'.format(results[k])
    results['ui'] = results['lb']+'-'+results['ub']
    results['mean_and_ui'] = f"""{results['mean']} (95% UI {results['ui']})"""
    return results

total_PEP = {}
for c,a in n_PEP.keys():
    total_PEP[c,a] = my_summary(pd.DataFrame(n_PEP[c,a]).sum())

In [ ]:
print(f"""In a hypothetical scenario where 50% of those exposed and of age 50 or older receive PEP,
we estimate {total_PEP[.5,50]['mean_and_ui']} individuals would be treated.""")

In [ ]:
#results paragraph 2
print(f"""A scenario providing PEP to 50% of unvaccinated household contacts age 50+ resulted in treatment of {total_PEP[.5,50]['mean_and_ui']} individuals, with fewer individuals treated at higher age thresholds (Table 1). PEP coverage of 50% of unvaccinated contacts age 50+ averted {symptomatic_infections_averted.loc[0.5, 50]} symptomatic COVID-19 cases, {hospitalizations_averted.loc[0.5, 50]} hospitalizations, and prevented {deaths_averted.loc[0.5, 50]} deaths. Expanding the age threshold to 20+ years increased the averted burden to {symptomatic_infections_averted.loc[0.5, 20]} symptomatic COVID-19 cases, {hospitalizations_averted.loc[0.5, 20]} hospitalizations, and {deaths_averted.loc[0.5, 20]} deaths, while shrinking the age threshold to age 80+ years reduced the burden averted to {symptomatic_infections_averted.loc[0.5, 80]} symptomatic COVID-19 cases, {hospitalizations_averted.loc[0.5, 80]} hospitalizations, and {deaths_averted.loc[0.5, 80]} deaths for the same assumption of 50% coverage of unvaccinated contacts in each target age group. Expanding PEP coverage by half from 50% to 75% resulted in corresponding 50% reductions in symptomatic COVID-19 cases, hospitalizations, and deaths.""")

In [ ]:
#result paragraph 3
print(f"""The rates of averted COVID-19 cases, hospitalization, and death differed by race and ethnicity. Assuming 50% mAbs PEP coverage of unvaccinated individuals age 50+, the estimated rate of averted secondary infections was {averted_symptomatic_race[0]}, {averted_symptomatic_race[1]}, {averted_symptomatic_race[3]}, and {averted_symptomatic_race[2]} symptomatic infections per 10,000,000 person-months among non-Hispanic Black, Hispanic, non-Hispanic white, and non-Hispanic members of other racial and ethnic groups, respectively.  The averted hospitalization rate per 10,000,000 person-months were {averted_hosp_race[0]}, {averted_hosp_race[1]}, {averted_hosp_race[3]}, and {averted_hosp_race[2]} while the averted death rates per 10,000,000 person-months were {averted_death_race[0]}, {averted_death_race[1]}, {averted_death_race[3]}, and {averted_death_race[2]} for the same races, respectively.""") 

In [ ]:
print(f"""While in a hypothetical scenario where 50% of those exposed and of age 80 or older receive PEP,
we estimate {total_PEP[.5,80]['mean']} (95% UI {total_PEP[.5,80]['ui']}) individuals would be treated,""")

In [ ]:
print(f"""and in a hypothetical scenario where 50% of those exposed and of age 20 or older receive PEP,
we estimate {total_PEP[.5,20]['mean']} (95% UI {total_PEP[.5,20]['ui']}) individuals would be treated.""")

In [ ]:
total_COVID = {}
for c,a in n_COVID.keys():
    total_COVID[c,a] = my_summary(pd.DataFrame(n_COVID[c,a]).sum())

In [ ]:
print(f"""[[Number who develop symptomatic COVID-19 in baseline and alternative scenarios, and for alternative age cutoffs. Including stratification by race and ethnicity.]]""")

print(f"""We estimate that without PEP, {total_COVID[0,0]['mean_and_ui']} individuals developed a symptomatic COVID-19 infection from a household exposure,
      while in a hypothetical scenario where 50% of those exposed and of age 50 or older receive PEP,
{total_COVID[.5,50]['mean_and_ui']} individuals developed a symptomatic infection.""")

In [ ]:
print(f"""In a hypothetical scenario where 50% of those exposed and of age 80 or older receive PEP,
we estimate {total_COVID[.5,80]['mean_and_ui']} individuals developed symptomatic infection,
and in a hypothetical scenario where 50% of those exposed and of age 20 or older receive PEP,
we estimate {total_COVID[.5,20]['mean_and_ui']} individuals developed symptomatic infection.""")

In [ ]:
total_hosp = {}
for c,a in n_hosp.keys():
    total_hosp[c,a] = my_summary(pd.DataFrame(n_hosp[c,a]).sum())
total_deaths = {}
for c,a in n_deaths.keys():
    total_deaths[c,a] = my_summary(pd.DataFrame(n_deaths[c,a]).sum())

In [ ]:
print(f"""We estimate that without PEP, {total_hosp[0,0]['mean_and_ui']} individuals would be hospitalized
because of a COVID-19 infection from a household exposure,
while in a hypothetical scenario where 50% of those exposed and of age 50 or older receive PEP,
{total_hosp[.5,50]['mean_and_ui']} individuals would be hospitalized;
in a scenario where 50% of those exposed and of age 80 or older receive PEP,
we estimate {total_hosp[.5,80]['mean_and_ui']} individuals would be hospitalized,
and in a scenario where 50% of those exposed and of age 20 or older receive PEP,
{total_hosp[.5,20]['mean_and_ui']} individuals would be hospitalized.
""")

In [ ]:
print(f"""We estimate that without PEP, {total_deaths[0,0]['mean_and_ui']} individuals would die
because of a COVID-19 infection from a household exposure,
while in a hypothetical scenario where 50% of those exposed and of age 50 or older receive PEP,
{total_deaths[.5,50]['mean_and_ui']} individuals would die;
in a scenario where 50% of those exposed and of age 80 or older receive PEP,
{total_deaths[.5,80]['mean_and_ui']} individuals would die,
and in a scenario where 50% of those exposed and of age 20 or older receive PEP,
{total_deaths[.5,20]['mean_and_ui']} individuals would die.
""")

In [ ]:
total_cost = {}
hosp_cost_str = {}
PEP_cost = {}
savings = {}
incremental_cost = {}

total_cost_0 = (pd.Series(mAbs_cost[0,0]) + pd.Series(hosp_cost[0,0])) / 1_000_000

for c,a in n_hosp.keys():
    total_cost_ca = (pd.Series(mAbs_cost[c,a]) + pd.Series(hosp_cost[c,a])) / 1_000_000
    total_cost[c,a] = my_summary(total_cost_ca)
    savings[c,a] = my_summary(total_cost_0 - total_cost_ca)
    incremental_cost[c,a] = my_summary(total_cost_ca - total_cost_0)
    hosp_cost_str[c,a] = my_summary(pd.Series(hosp_cost[c,a])/1_000_000)
    PEP_cost[c,a] = my_summary(pd.Series(mAbs_cost[c,a])/1_000_000)

hosp_cost_str[.5,20] , PEP_cost[.5,20]

In [ ]:
savings[0.5,50]

In [ ]:
#results paragraph 3
print(f"""We estimate that without PEP, the cost of hospitalizations due to COVID-19 infections from household exposure would be {hosp_cost_str[0,0]['mean_and_ui']} million dollars, while in a hypothetical scenario where 50% of unvaccinated household contacts age 50 or older receive PEP, the cost of hospitalizations would be {hosp_cost_str[.5,50]['mean_and_ui']} million dollars and the cost of PEP would be {PEP_cost[.5,50]['mean_and_ui']} million dollars, for a total of {total_cost[.5,50]['mean_and_ui']} million dollars, which is a savings of {savings[.5,50]['mean_and_ui']} million dollars compared to the without-PEP scenario. In a scenario where 50% of those exposed and of age 80 or older receive PEP, the cost of hospitalizations would be {hosp_cost_str[.5,80]['mean_and_ui']} million dollars and the cost of PEP would be {PEP_cost[.5,80]['mean_and_ui']} million dollars, for a total of {total_cost[.5,80]['mean_and_ui']} million dollars, which is a savings of {savings[.5,80]['mean_and_ui']} million dollars compared to the without-PEP scenario. Alternatively, in a scenario where 50% of those exposed and of age 20 or older receive PEP, the cost of hospitalizations would be {hosp_cost_str[.5,20]['mean_and_ui']} million dollars and the cost of PEP would be {PEP_cost[.5,20]['mean_and_ui']} million dollars, for a total of {total_cost[.5,20]['mean_and_ui']} million dollars, which is a total of {incremental_cost[.5,20]['mean_and_ui']} million dollars more than in the without-PEP scenario. The incremental cost and COVID-19 deaths averted for a range of scenarios is summarized in Figure 2.""")

In [ ]:
total_cost[c,a]

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(6.5,3), dpi=120)
for c in [0.25, .5, .75]:
    for i, a in enumerate(age_lower_bounds[::-1]):
        s_cost = (pd.Series(mAbs_cost[c,a]) + pd.Series(hosp_cost[c,a])) / 1e6
        s_cost -= (pd.Series(mAbs_cost[0,0]) + pd.Series(hosp_cost[0,0])) / 1e6
        
        s_deaths = pd.DataFrame(n_deaths[c,a]).sum(axis=0)
        s_deaths -= pd.DataFrame(n_deaths[0,0]).sum(axis=0)
        s_deaths *= -1
        
        if c == 0.5:
            plt.plot(s_deaths, s_cost, 'o', label=f'{a}+', alpha=.95, color='none', mec=f'C{i}', mew=1)
#         else:
#             plt.plot(s_deaths, s_cost, 'o', color='grey')
plt.legend(loc=(1.02, .02), title='50% coverage, with\nPEP for exposed of age:')
plt.xlabel('COVID-19 deaths averted (people)')
plt.ylabel('Incremental cost (millions of dollars)')
plt.grid()
plt.subplots_adjust(right=.7);
plt.savefig('output.png', dpi=300, bbox_inches='tight')


In [ ]:
#Table 1
#nvaccinated household contacts n_2.sum().mean(), (np.percentile(n_2.sum(), 2.5),np.percentile(n_2.sum(), 97.5))
def m_ui(z):
    return f'{np.mean(z):,.0f} ({np.percentile(z, 2.5):,.0f} - {np.percentile(z, 97.5):,.0f})'

def contacts(df,age_range):
    results = m_ui(pd.DataFrame(df).unstack(0).sum().unstack(0).loc[age_range:].sum())
    return results


tab1 = {'Age threshold for PEP': ['Baseline (no PEP)', '80+', '50+', '20+'],
        'Household contacts': [contacts(n_2_all,0), contacts(n_2_all,80), contacts(n_2_all,50), contacts(n_2_all,20)],
        'Unvaccinated household contacts': [contacts(n_2_unvax,0), contacts(n_2_unvax,80), contacts(n_2_unvax,50), contacts(n_2_unvax,20)],
        'PEP coverage': ['0', '50%', '50%', '50%'],
        'Number Treated with PEP': ['0', total_PEP[.5, 80]['mean_and_ui'], total_PEP[.5,50]['mean_and_ui'], total_PEP[.5,20]['mean_and_ui']],
        'Symptomatic COVID-19 from household exposure': [total_COVID[0, 0]['mean_and_ui'], total_COVID[.5,80]['mean_and_ui'] , total_COVID[.5,50]['mean_and_ui'], total_COVID[.5,20]['mean_and_ui']],
        'Hospitalizations from household exposure': [total_hosp[0, 0]['mean_and_ui'], total_hosp[.5, 80]['mean_and_ui'] , total_hosp[.5,50]['mean_and_ui'], total_hosp[.5,20]['mean_and_ui']],
        'Deaths from household exposure': [total_deaths[0, 0]['mean_and_ui'], total_deaths[.5, 80]['mean_and_ui'], total_deaths[.5,50]['mean_and_ui'], total_deaths[.5,20]['mean_and_ui']]}


pd.DataFrame(tab1, columns = ['Household contacts',
                              'Unvaccinated household contacts',
                              'PEP coverage',
                              'Number Treated with PEP',
                              'Symptomatic COVID-19 from household exposure', 
                              'Hospitalizations from household exposure', 
                              'Deaths from household exposure'], 
            index = ['Baseline (no PEP)', '80+', '50+', '20+']).apply(lambda col: col.str.replace("95% UI ", ""))

In [ ]:
df_cdc_month.race_eth.value_counts(normalize=True).mul(100).round(2).astype(str)+%

In [ ]:
def contacts(df,age):
    age = m_ui(pd.DataFrame(df).unstack(0).sum().unstack(0).loc[age:].sum())
    return age


In [ ]:
df_cdc_month.race_eth.value_counts()

In [ ]:
len(df_cdc_month)

In [ ]:
dd = len(df_cdc_month) - df_cdc_month.race_eth.isna().sum() #Non-NA total (removing NAs)
df_cdc_month.race_eth.value_counts().div(dd).mul(100).round(2).astype(str)+'%'

In [ ]:
n_PEP_mean.unstack(1).unstack().sum().sum()